# Cuarto Notebook

En el siguiente notebook se entrenar un modelo YOLO usando transfer learning a partir de pesos
preentrenados, utilizando el dataset preparado en formato YOLO (Notebook 02) el anterior xd.

Este notebook:
- Carga un modelo YOLO preentrenado
- Entrena solo con las clases: car, airplane, truck
- Usa un número controlado de épocas
- Guarda el mejor modelo (`best.pt`)
- Deja el entrenamiento listo para ser rastreado luego con MLflow


In [1]:
# --- IMPORTS ---
from pathlib import Path
import json


In [2]:
# --- RUTAS DEL PROYECTO ---
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

YOLO_DATASET_DIR = PROJECT_ROOT / "data" / "processed" / "yolo_dataset"
DATA_YAML_PATH = YOLO_DATASET_DIR / "data.yaml"

MODELS_DIR = PROJECT_ROOT / "models"
RUNS_DIR = PROJECT_ROOT / "runs"
ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"

CONFIG_SNAPSHOT_PATH = ARTIFACTS_DIR / "config_snapshot.json"

MODELS_DIR.mkdir(parents=True, exist_ok=True)
RUNS_DIR.mkdir(parents=True, exist_ok=True)

print("Dataset:", DATA_YAML_PATH)


Dataset: c:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset\data.yaml


In [3]:
# --- CARGA DE CONFIGURACIÓN DEL PROYECTO ---
#carga de configuracion previa batch, epoch para el entrenamiento
with open(CONFIG_SNAPSHOT_PATH, "r", encoding="utf-8") as f:
    cfg = json.load(f)

img_size = cfg["img_size"]
batch_size = cfg["batch_size"]
epochs = cfg["epochs"]
seed = cfg["seed"]

print("img_size:", img_size)
print("batch_size:", batch_size)
print("epochs:", epochs)
print("seed:", seed)


img_size: 640
batch_size: 16
epochs: 20
seed: 42


In [4]:
# --- INSTALACIÓN / IMPORT DE YOLO (ULTRALYTICS) ---

try:
    from ultralytics import YOLO
    print("Ultralytics YOLO importado correctamente.")
except Exception as e:
    raise RuntimeError(
        "Ultralytics YOLO no está instalado.\n"
        "Instala con: pip install ultralytics"
    ) from e


Ultralytics YOLO importado correctamente.


In [5]:
# --- SELECCIÓN DEL MODELO BASE (TRANSFER LEARNING) ---
# Opciones típicas: yolov8n.pt (rápido), yolov8s.pt (mejor accuracy)
YOLO_BASE_MODEL = "yolov8n.pt"

print("Modelo base:", YOLO_BASE_MODEL)


Modelo base: yolov8n.pt


In [6]:
# --- INICIALIZACIÓN DEL MODELO ---
#el modelo base de entrebamiento que sera utilizado mas adelante
model = YOLO(YOLO_BASE_MODEL)


In [7]:
# --- MLFLOW URI FIX (WINDOWS) ---
#Existe un error en windws con respeto a las rutas que usara el tracking de mlflow para solucionarlo se debe apuntar a la carpeta de dependecias del mlflow creada anteriormente
import os
from pathlib import Path

MLRUNS_DIR = (Path.cwd().parent / "mlruns").resolve()
MLRUNS_DIR.mkdir(parents=True, exist_ok=True)

tracking_uri = MLRUNS_DIR.as_uri()  

os.environ["MLFLOW_TRACKING_URI"] = tracking_uri
os.environ["MLFLOW_REGISTRY_URI"] = tracking_uri

print("MLFLOW_TRACKING_URI =", os.environ["MLFLOW_TRACKING_URI"])
print("MLFLOW_REGISTRY_URI =", os.environ["MLFLOW_REGISTRY_URI"])


MLFLOW_TRACKING_URI = file:///C:/Users/Johnny/Desktop/IA/mlruns
MLFLOW_REGISTRY_URI = file:///C:/Users/Johnny/Desktop/IA/mlruns


In [8]:
# --- ENTRENAMIENTO ---
#aqui solo se carga los parametros del entrenamiento definidos en el primer notebook 
train_results = model.train(
    data=str(DATA_YAML_PATH),
    imgsz=img_size,
    batch=batch_size,
    epochs=epochs,
    seed=seed,
    project=str(RUNS_DIR),
    name="yolo_coco_subset",
    pretrained=True,
    optimizer="SGD",
    lr0=0.01,
    patience=5,
    verbose=True
)


New https://pypi.org/project/ultralytics/8.4.9 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.8  Python-3.12.10 torch-2.9.1+cpu CPU (Intel Core 7 150U)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=

c:\Users\Johnny\AppData\Local\Python\pythoncore-3.12-64\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)


MLflow: logging run_id(bc0cab4bfbc6405584bd734a93f48d27) to file:///C:/Users/Johnny/Desktop/IA/mlruns
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\Johnny\Desktop\IA\runs\yolo_coco_subset7
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/20         0G      1.273      2.481      1.223         33        640: 100% ━━━━━━━━━━━━ 195/195 7.2s/it 23:205.7ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 18/18 5.0s/it 1:295.4sss
                   all        550       1895      0.547      0.473      0.485      0.307

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/20         0G      1.347      1.885      1.284         41        640: 100% ━━━━━━━━━━━━ 195/195 7.0s/it 22:465.5ss
                 Class     Images  Instances      Box(P   

In [9]:
# --- UBICACIÓN DEL MEJOR MODELO (DINÁMICA) ---
from pathlib import Path

save_dir = Path(model.trainer.save_dir)   
BEST_MODEL_PATH = save_dir / "weights" / "best.pt"

if not BEST_MODEL_PATH.exists():
    raise FileNotFoundError(f"No se encontró best.pt. Revisar: {save_dir}")

print("Best model:", BEST_MODEL_PATH)


Best model: C:\Users\Johnny\Desktop\IA\runs\yolo_coco_subset7\weights\best.pt


In [10]:
# --- COPIAR MODELO FINAL A /models ---
#Una vez finalizado el entrenamiento y la seleccion del mejor modelo se guarda para ser utilizado despues
# --- COPIAR MODELO FINAL A /models ---
FINAL_MODEL_PATH = MODELS_DIR / "yolo_best.pt"
FINAL_MODEL_PATH.write_bytes(BEST_MODEL_PATH.read_bytes())

print("Modelo final copiado a:", FINAL_MODEL_PATH)



Modelo final copiado a: c:\Users\Johnny\Desktop\IA\models\yolo_best.pt


In [11]:
# --- EVALUACIÓN RÁPIDA (VAL) ---
#comrueba una vez mas si el modeo seleccionado es el mejor y si ccumplio con todos los parametros establecidos anetriormente
metrics = model.val(
    data=str(DATA_YAML_PATH),
    imgsz=img_size,
    batch=batch_size,
    conf=cfg["conf_threshold"],
    iou=cfg["iou_threshold"]
)

print("Evaluación completada.")


Ultralytics 8.4.8  Python-3.12.10 torch-2.9.1+cpu CPU (Intel Core 7 150U)
Model summary (fused): 73 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 415.6111.4 MB/s, size: 141.5 KB)
val: Scanning C:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset\labels\val.cache... 550 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 550/550  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 1.0it/s 34.2s1.0ss
                   all        550       1895      0.571      0.485      0.536      0.361
                   car        378       1337       0.52        0.5      0.516      0.332
              airplane         97        143      0.808      0.587      0.747      0.522
                 truck        250        415      0.385      0.369      0.344      0.229
Speed: 0.5ms preprocess, 53.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to C:\Users\Johnny\r

In [12]:
# --- RESUMEN FINAL ---
print("Entrenamiento finalizado.")
print("Modelo guardado en:", FINAL_MODEL_PATH)
print("Runs en:", RUNS_DIR / "yolo_coco_subset")


Entrenamiento finalizado.
Modelo guardado en: c:\Users\Johnny\Desktop\IA\models\yolo_best.pt
Runs en: c:\Users\Johnny\Desktop\IA\runs\yolo_coco_subset


In [13]:
# --- GUARDAR RUTA DEL RUN PARA EL NOTEBOOK 04 ---
from pathlib import Path
import json

run_dir = Path(model.trainer.save_dir)
run_info_path = ARTIFACTS_DIR / "last_run.json"

run_info = {"yolo_run_dir": str(run_dir)}
run_info_path.write_text(json.dumps(run_info, indent=2), encoding="utf-8")

print("Guardado last_run.json en:", run_info_path)
print("Run dir:", run_dir)


Guardado last_run.json en: c:\Users\Johnny\Desktop\IA\artifacts\last_run.json
Run dir: C:\Users\Johnny\Desktop\IA\runs\yolo_coco_subset7
